<a href="https://colab.research.google.com/github/MasahiroAraki/MachineLearning3/blob/master/notebook/chap14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U scikit-learn keras --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.5 MB/s eta 0:00:00


# 第14章 半教師あり学習



## 例題 14.1

Iris データを使って自己学習を行え．正解なしデータの割合を変えて，性能の変化を確認せよ．

ライブラリの読み込み

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.semi_supervised import SelfTrainingClassifier, LabelSpreading

In [2]:
%precision 3
np.set_printoptions(precision=3, suppress=True)

Irisデータの読み込みと学習用・評価用の分割

In [3]:
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
y_test

array([0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 1, 1,
       2, 1, 1, 0, 0, 2, 0, 2, 2, 0, 1, 2, 1, 0, 2, 1, 1, 2, 1, 1, 2, 1,
       0, 2, 0, 1, 0, 0])

### 半教師ありデータの作成

まず、正解データ`y_train`の2/3がTrueとなる真偽値のインデックスを作ります。そして`y_train`を[np.copy](https://docs.scipy.org/doc/numpy/reference/generated/numpy.copy.html)を使って変数labelsにコピーし（代入文でコピーするとオブジェクトが共有され、labelsを変更するとyも変わってしまいます）、真偽値インデックスでTrueに対応するラベルの値を「ラベルなし」を意味する「-1」に書き換えます。

In [4]:
rng = np.random.default_rng(2)

def make_unlabeled(y, ratio):
    n = len(y)
    k = int(n * ratio)
    idx = rng.choice(n, size=k, replace=False)
    labels = np.copy(y)
    for i in idx:
        labels[i] = -1
    return labels

labels = make_unlabeled(y_train, 0.7)
labels

array([ 0,  2, -1, -1,  0,  2,  2,  2, -1, -1, -1,  2,  1, -1, -1, -1, -1,
        0, -1,  0,  2,  1,  1, -1, -1, -1, -1,  2,  0, -1, -1,  1, -1, -1,
        1,  0,  0, -1, -1, -1,  0, -1, -1, -1,  1, -1,  1, -1, -1, -1,  2,
       -1, -1, -1,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  2, -1,  2,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  2,  1, -1, -1, -1,  0])

In [5]:
from collections import Counter
print(Counter(labels.tolist()))

Counter({-1: 70, 2: 12, 0: 9, 1: 9})


これを教師ベクトルとして、[SelfTrainingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.SelfTrainingClassifier.html)で半教師あり学習を行います。識別器には[SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) （デフォルトのRBFカーネル）を識別結果に確率を付けるように、かつハイパーパラメータgammaの設定から特徴ベクトルの分散を除外するようにして使います。後者は、特徴ベクトルの分散こそが正解なしデータから得たいものだからです。

In [6]:
svc = SVC(probability=True, gamma="auto", random_state=3)
clf = SelfTrainingClassifier(svc, verbose=True)
clf.fit(X_train, labels)

End of iteration 1, added 49 new labels.
End of iteration 2, added 9 new labels.
End of iteration 3, added 2 new labels.
End of iteration 4, added 1 new labels.


,estimator,SVC(gamma='au...andom_state=3)
,base_estimator,'deprecated'
,threshold,0.75
,criterion,'threshold'
,k_best,10
,max_iter,10
,verbose,True
,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'auto'


性能を評価します

In [7]:
clf.score(X_test, y_test)

0.960

正解なしデータの割合を変えて，性能を評価します．学習用と評価用の分割や，正解の有無のパターンを変えて，それぞれの割合で100回試行して，平均を出力しています．

In [8]:
unlabeled_percent = [0.9, 0.8, 0.7, 0.6, 0.5]
for p in unlabeled_percent :
    score = 0
    for _ in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4)
        labels = make_unlabeled(y_train, p)
        svc = SVC(probability=True, gamma="auto", random_state=rng.integers(100))
        clf = SelfTrainingClassifier(svc, verbose=False)
        clf.fit(X_train, labels)
        score += clf.score(X_test, y_test)
    print(f'unlabeled:{p*100:4.1f}%, score={score/100:6.3f}')

unlabeled:90.0%, score= 0.807
unlabeled:80.0%, score= 0.957
unlabeled:70.0%, score= 0.973
unlabeled:60.0%, score= 0.979
unlabeled:50.0%, score= 0.978


## 例題14.2

`LabelSpreading` アルゴリズムで， Iris データを用いて，正解なしデータのラベル予測を行え．



scikit-learnの[LabelSpreading](https://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelSpreading.html)でirisデータの半教師あり学習を行います。ここでは、すべてのデータを使い、正解なしデータの予測精度で評価します。

In [9]:
rng = np.random.default_rng(2)

labels = make_unlabeled(y, 0.7)
lp = LabelSpreading(max_iter=10000)
lp.fit(X, labels)

,kernel,'rbf'
,gamma,20
,n_neighbors,7
,alpha,0.2
,max_iter,10000
,tol,0.001
,n_jobs,None


正解なしデータの割り当て結果を表示します。

In [10]:
unlabeled = labels == -1
lp.score(X[unlabeled], y[unlabeled])

0.933

正解付きデータの割合を5%, 10%, 20%, 30%と変えて、それぞれ100回ずつ学習を試みて性能を評価します。

In [11]:
unlabeled_percent = [0.95, 0.9, 0.8, 0.7]

for p in unlabeled_percent :
    score = 0
    for _ in range(100):
        labels = make_unlabeled(y, p)
        lp = LabelSpreading(max_iter=10000)
        lp.fit(X, labels)
        unlabeled = labels == -1
        score += lp.score(X[unlabeled], y[unlabeled])
    print(f'unlabeled:{p:3.0%}, score={score/100:6.3f}')

unlabeled:95%, score= 0.865
unlabeled:90%, score= 0.923
unlabeled:80%, score= 0.948
unlabeled:70%, score= 0.952


## 演習問題

14.1 `LabelPropagation` で，Breast Cancer データを用いて，正解なしデータのラベル予測を行え．その際，正解付きデータの割合を変化させて，性能を評価すること．

例題14.1, 14.2 の環境を使います

In [15]:
from sklearn.datasets import load_breast_cancer
from sklearn.semi_supervised import LabelPropagation
from sklearn.preprocessing import normalize

bc = load_breast_cancer()
X = bc.data
y = bc.target

rng = np.random.default_rng(2)
X = normalize(X)
labeled_percent = [0.05, 0.1, 0.2, 0.3, 0.5]
for labeled in labeled_percent:
    score = 0
    for i in range(100):
        labels = make_unlabeled(y, 1-labeled)
        lp = LabelPropagation(gamma=1000)
        lp.fit(X, labels)
        unlabeled = labels == -1
        score += lp.score(X[unlabeled], y[unlabeled])
    print(f'labeled:{labeled*100:4.1f}%, score={score/100:6.3f}')

labeled: 5.0%, score= 0.777
labeled:10.0%, score= 0.850
labeled:20.0%, score= 0.883
labeled:30.0%, score= 0.896
labeled:50.0%, score= 0.909
